In [1]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', DEVICE)
print()

Using device: cuda



In [2]:
model_name = "cointegrated/rut5-small-chitchat"

tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [3]:
from transformers import TextDataset, DataCollatorForLanguageModeling

train_path = 'dataset/abatur_train.csv'

# Создание датасета
train_dataset = TextDataset(tokenizer=tokenizer,file_path=train_path,block_size=64)

# Создание даталодера (нарезает текст на оптимальные по длине куски)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

C:\Users\konto\anaconda3\envs\PythonGPU\lib\site-packages\transformers\data\datasets\language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [4]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./finetuned", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=2000, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=32,  # batch size for evaluation
    warmup_steps=10,# number of warmup steps for learning rate scheduler
    gradient_accumulation_steps=16, # to make "virtual" batch size larger
    )



trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5),None) # Optimizer and lr scheduler
)

In [ ]:
trainer.train()
trainer.save_model("./final_save")

In [31]:
text = 'В чем заключается твоя работа ?'
inputs = tokenizer(text, return_tensors='pt').to(DEVICE)
with torch.no_grad():
    gen = model.generate(
        **inputs, 
        do_sample=True, top_p=0.5, num_return_sequences=1, 
        repetition_penalty=2.5,
        max_length=64,
    )
for h in gen:
    print(tokenizer.decode(h, skip_special_tokens=True))

Работа заключается в том, что ты имеешь право на свою жизнь.


In [39]:
text = 'В чем заключается твоя работа ?'
inputs = tokenizer(text, return_tensors='pt').to(DEVICE)
with torch.no_grad():
    hypotheses = model.generate(
        **inputs, 
        do_sample=True, temperature=1.1,
                     max_length=64)
    
for h in hypotheses:
    print(tokenizer.decode(h, skip_special_tokens=True))

Работа заключается в этом. Вселенной


In [35]:
# Сэмплирование с температурой
text = "В чем заключается твоя работа ? "
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                     do_sample=True, 
                     temperature=1.1,
                     max_length=64) 
                     
# 1.4
# 1.6
generated_text = list(map(tokenizer.decode, out))[0]

print(generated_text)

<pad> В том, чтобы я начал бы с этим. Работа, которая заключается в них.</s>
